In [10]:
!pip install qiskit==0.43.3

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for qiskit-terra==0.24.2 from https://files.pythonhosted.org/packages/ee/51/f863a3d36d3320926156514a7be2f0ce36d78ce80e7508b8f140914dface/qiskit_terra-0.24.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for qiskit-aer==0.12.2 from https://files.pythonhosted.org/packages/51/9d/0098c7d1c1914251d3e2ecaa9dd512639bc0cdc1f635597726845e66f4df/qiskit_aer-0.12.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for symengine<0.10,>=0.9 from https://files.pythonhosted.org/packages/a5/68/98845ba17019a9c8e7de19f037ac4e7dfd81a871119466bf5d3045c4bd18/symengine-0.9.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached symengine-0.9.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.5 MB/s eta 0:00:00a 0:00:01


In [11]:
"""
Simplified ASP Solver for Mining Problems

This module provides a simplified approach to solving open-pit mining problems
using Adiabatic State Preparation (ASP), combining functionality from the Mine
and ASP classes into a single function.
"""

import numpy as np
from typing import Optional, Callable, Union, Tuple, List, Dict
from collections import defaultdict
# from qiskit.quantum_info import SparsePauliOp
# from qiskit.circuit.library import PauliEvolutionGate
# from qiskit.primitives import Estimator, Sampler
# from qiskit_aer.primitives import Estimator as AerEstimator
# from qiskit_aer.primitives import Sampler as AerSampler
# from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.opflow import X
from qiskit.opflow import Z
from qiskit.opflow import I
from qiskit.opflow import PauliOp
from qiskit.opflow import Plus  # Backward compatibility
# from qiskit.transpiler import PassManager
from time import perf_counter

def solve_asp(
    mine_config: Union[str, np.ndarray],
    evol_time: float = 10,
    nsteps: int = 20,
    callback: Optional[Callable[[QuantumCircuit, int], None]] = None,
    callback_freq: int = 1,
    quantum_instance: Optional[Union[QuantumInstance, BaseBackend, Backend]] = None,
    penalty: Optional[Union[float, bool]] = None,
    benchmark: bool = False
) -> Dict:
    """
    Solve an open-pit mining problem using Adiabatic State Preparation (ASP).
    
    Parameters
    ----------
    mine_config : Union[str, np.ndarray]
        Path to the mine configuration file or a numpy array with the mine data.
    evol_time : float, optional
        Time for the adiabatic evolution.
    nsteps : int, optional
        Number of discrete time blocks for Trotterization.
    callback : Callable, optional
        A callback function that has access to the partially built circuit.
    callback_freq : int, optional
        The frequency in which the callback function is called.
    quantum_instance : Union[QuantumInstance, BaseBackend, Backend], optional
        Quantum instance to run the circuit.
    penalty : Union[float, bool], optional
        Penalty for the smoothness term in the Hamiltonian.
        If True, a heuristic penalty will be calculated.
    benchmark : bool, optional
        Whether to enable benchmarking.
        
    Returns
    -------
    Dict
        A dictionary containing the solution results, including the optimal configuration,
        probability, profit, violation (if applicable), and timing information.
    """
    # Start benchmarking if enabled
    start_time = perf_counter() if benchmark else None
    
    # Initialize quantum instance if not provided
    if quantum_instance is None:
        quantum_instance = QasmSimulator(method="statevector")
    if not isinstance(quantum_instance, QuantumInstance):
        quantum_instance = QuantumInstance(quantum_instance)
    
    # Parse mine configuration
    if isinstance(mine_config, str):
        try:
            dat = np.loadtxt(mine_config, dtype=float)
        except:
            raise IOError("Invalid Mine Configuration File")
    elif isinstance(mine_config, np.ndarray):
        if len(mine_config.shape) != 2:
            raise ValueError("`mine_config` must be two-dimensional")
        dat = np.array(mine_config, dtype=float)
    else:
        raise ValueError("Unrecognized `mine_config` type")
    
    # Extract mine dimensions and initialize graph structure
    rows, cols = dat.shape
    graph = defaultdict(list)  # p:c (parent to child)
    graph_r = defaultdict(list)  # c:p (child to parent)
    idx2cord = []
    cord2idx = {}
    
    # Initialize mapping
    for r in range(rows):
        for c in range(cols):
            if c < r or c > cols - r - 1:
                dat[r, c] = float("inf")
            if dat[r, c] < float("inf"):
                idx2cord.append((r, c))
                cord2idx[(r, c)] = len(idx2cord) - 1
                idx = cord2idx[(r, c)]
                for pr, pc in [(r - 1, c - 1), (r - 1, c), (r - 1, c + 1)]:
                    if (0 <= pr < rows and 0 <= pc < cols and 
                        dat[pr, pc] < float("inf")):
                        graph[idx].append(cord2idx[(pr, pc)])
                        graph_r[cord2idx[(pr, pc)]].append(idx)
    
    nqubits = len(idx2cord)
    
    # Generate Hamiltonian components
    def gen_Hs() -> PauliOp:
        """Generate the smoothness Hamiltonian."""
        Hs = 0 * I ^ nqubits
        for i in range(nqubits):
            for j in graph[i]:
                Hs += ((I ^ nqubits) - ((I ^ (nqubits - i - 1)) ^ Z ^ (I ^ i))) @ ((I ^ nqubits) + ((I ^ (nqubits - j - 1)) ^ Z ^ (I ^ j)))
        return 0.25 * Hs
    
    def gen_Hp() -> PauliOp:
        """Generate the profit Hamiltonian."""
        Hp = 0 * I ^ nqubits
        for i in range(nqubits):
            Hp += float(dat[idx2cord[i]]) * ((I ^ nqubits) - ((I ^ (nqubits - i - 1)) ^ Z ^ (I ^ i)))
        return 0.5 * Hp
    
    # Calculate Hamiltonians
    Hs = gen_Hs()
    Hp = gen_Hp()
    
    # Calculate heuristic penalty if needed
    def heuristic_penalty(coeff: float = 3.8) -> float:
        return float(np.linalg.norm(np.where(dat.flat != np.inf), ord=2) / nqubits) * coeff
    
    # Apply penalty if specified
    if penalty is True:
        penalty = heuristic_penalty()
    
    # Generate the full Hamiltonian
    if penalty:
        hamiltonian = (-Hp + penalty * Hs).reduce()
    else:
        # Generate the projected Hamiltonian
        state_fn = (-Hs @ (Plus ^ nqubits)).reduce().eval().to_dict_fn()
        valid_configs = [int(k, 2) for k, v in state_fn.primitive.items() if abs(v) < 1e-8]
        p_op = np.zeros((2 ** nqubits))
        p_op[np.array(valid_configs, dtype=int)] = 1
        from qiskit.opflow import MatrixOp
        p_op = MatrixOp(np.diag(p_op))
        hamiltonian = (p_op @ -Hp @ p_op).reduce().to_matrix_op()
    
    # Set up the initial state and operator for ASP
    initial_state = (Z @ I ^ (nqubits - 1)).neg().to_circuit()
    initial_operator = X ^ (I ^ (nqubits - 1))
    for i in range(1, nqubits):
        initial_operator += (I ^ i) ^ X ^ (I ^ (nqubits - i - 1))
    
    # Construct the ASP circuit
    circuit = initial_state.copy()
    for i in range(nsteps):
        xi = (0.5 + i) / nsteps
        circuit.hamiltonian(
            hamiltonian,
            xi * evol_time / nsteps,
            list(range(nqubits)),
        )
        circuit.hamiltonian(
            initial_operator,
            (1 - xi) * evol_time / nsteps,
            list(range(nqubits)),
        )
        if callback and i % callback_freq == 0:
            callback(circuit, i)
    
    # Execute the circuit
    if quantum_instance.is_statevector:
        result = quantum_instance.execute(circuit)
        eigenstate = result.get_statevector(circuit)
        # Find the most probable state
        probs = np.abs(eigenstate) ** 2
        max_idx = np.argmax(probs)
        optimal_config = format(max_idx, f"0{nqubits}b")
        optimal_config_prob = probs[max_idx]
    else:
        result = quantum_instance.execute(circuit.measure_all(inplace=False))
        counts = result.get_counts()
        optimal_config, count = max(counts.items(), key=lambda item: item[1])
        optimal_config_prob = count / quantum_instance.run_config.shots
    
    # Calculate profit and violation for the optimal configuration
    def get_profit(bitstring: str) -> float:
        """Return profit for a vector state."""
        return sum([dat[idx2cord[i]] for i in range(nqubits) if bitstring[i] == "1"])
    
    def get_violation(bitstring: str) -> int:
        """Return violation for a vector state."""
        dig = list(map(lambda x: -1 if x == "1" else 1, list(bitstring)))
        res = 0
        for i in range(nqubits):
            for j in graph[i]:
                res += 0.25 * (1.0 - dig[i]) * (1.0 + dig[j])
        return int(res)
    
    profit = get_profit(optimal_config)
    violation = get_violation(optimal_config)
    
    # Display the solution
    def print_mine_state(bitstring: str) -> None:
        """Print a visual representation of the mining state."""
        try:
            from prettytable import PrettyTable
            x = PrettyTable([" "] + [str(ic) for ic in range(cols)])
            for ir in range(rows):
                x.add_row(
                    [ir] + [
                        bitstring[cord2idx[(ir, ic)]]
                        if (ir, ic) in cord2idx
                        else "x"
                        for ic in range(cols)
                    ]
                )
            print(str(x))
        except ImportError:
            # Fallback if PrettyTable is not available
            for ir in range(rows):
                row = [
                    bitstring[cord2idx[(ir, ic)]]
                    if (ir, ic) in cord2idx
                    else "x"
                    for ic in range(cols)
                ]
                print(f"{ir}: {' '.join(row)}")
    
    print(f"The most probable configuration: {optimal_config}")
    print(f"Probability: {optimal_config_prob}")
    print(f"Profit: {profit}")
    print(f"Violation: {violation}")
    print_mine_state(optimal_config)
    
    # Finalize benchmarking
    duration = perf_counter() - start_time if benchmark else None
    
    # Return results
    return {
        "optimal_config": optimal_config,
        "optimal_config_prob": optimal_config_prob,
        "profit": profit,
        "violation": violation,
        "execution_time": duration,
        "circuit": circuit,
    }

ImportError: cannot import name 'BasicAer' from 'qiskit' (/Users/vivekchaudhary/anaconda3/lib/python3.11/site-packages/qiskit/__init__.py)

In [ ]:
import numpy as np
from qiskit.providers.aer import QasmSimulator

# Define mine configuration
mine_data = np.array([
    [-2.0, 3.0, -1.0, -2.0, -1.0],
    [float('inf'), 1.0, -5.0, 10.0, float('inf')],
    [float('inf'), float('inf'), 4.0, float('inf'), float('inf')]
])

# Define callback function (optional)
def analysis(circuit, iteration):
    print(f"Circuit at iteration {iteration} has depth {circuit.depth()}")

# Solve the mining problem with ASP
result = solve_asp(
    mine_data,
    evol_time=10,
    nsteps=20,
    callback=analysis,
    callback_freq=5,
    quantum_instance=QasmSimulator(),
    benchmark=True
)

# Access results
print(f"Best configuration: {result['optimal_config']}")
print(f"Profit: {result['profit']}")